In [2]:
# Oct 2024
# author: SBN

In [3]:
# fine-tuning the model by own dataset through unsloth
# The dataset is captured from PubMed articles (abstract texts)
# and curated into a clean and reliable dataset for training.
# the topic selected for fetching data from PubMed is 'skin cancer'.
# this Model when train can synthsis a skin cancer specialist and will answer your questions.

In [1]:
# This notebook was inspired by 
# https://github.com/amrs-tech/csv-data-gen-finetuned/blob/main/AMRS_Llama_3_1_8b_%2B_Unsloth_2x_faster_finetuning.ipynb
# https://www.kaggle.com/code/tahmidmir/fine-tuning-gpt-2-on-skin-cancer-articles

In [5]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset, Dataset
import pandas as pd
import json, yaml
import torch

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.45.2.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.615 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
# inpput_file = '/home/latso820/Documents/pypractice/Matariki/input.json'
inpput_file = '/home/latso820/Documents/pypractice/Matariki/PubMedAbstracts.json'



In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset, Dataset
import pandas as pd
import json, yaml
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
import pandas as pd
import yaml
from datasets import Dataset

# Open and load the JSON file
with open(inpput_file, 'r') as f:
    json_f = yaml.safe_load(f.read())

# Convert the list of dictionaries to a DataFrame directly
df = pd.DataFrame(json_f)

# Check the columns of the DataFrame to ensure it contains instruction, input, and output
print(df.columns)

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Print the loaded dataset
print('loaded data = ', dataset, type(dataset))

# Apply formatting or processing on the dataset using a map function
dataset = dataset.map(formatting_prompts_func, batched=True)
print('prcs loaded data = ', dataset, type(dataset))
     

Index(['instruction', 'input', 'output'], dtype='object')
loaded data =  Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 72
}) <class 'datasets.arrow_dataset.Dataset'>


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

prcs loaded data =  Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 72
}) <class 'datasets.arrow_dataset.Dataset'>


In [21]:

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 150,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/home/latso820/.local/lib/python3.10/site-packages/transformers/training_args.py:2027: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/72 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [22]:

#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A4000. Max memory = 15.615 GB.
6.895 GB of memory reserved.


In [23]:
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 72 | Num Epochs = 17
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 150
 "-____-"     Number of trainable parameters = 41,943,040


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


Step,Training Loss
10,1.089300
20,0.647200
30,0.391600
40,0.224200
50,0.128800
60,0.072800
70,0.064300
80,0.048900
90,0.038000
100,0.035700


In [24]:

#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

316.7699 seconds used for training.
5.28 minutes used for training.
Peak reserved memory = 10.373 GB.
Peak reserved memory for training = 3.478 GB.
Peak reserved memory % of max memory = 66.43 %.
Peak reserved memory for training % of max memory = 22.273 %.


In [27]:
# 

In [31]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a skin specialist and know everything about skin cancer", # instruction
        "What is mycosis fungoides?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a skin specialist and know everything about skin cancer\n\n### Input:\nWhat is mycosis fungoides?\n\n### Response:\nMycosis fungoides (MF) is a rare, slow-growing type of cutaneous lymphoma that typically presents as a persistent, itchy patch on the skin. It is the most common primary cutaneous lymphoma and is often diagnosed after a biopsy. Treatment options include topical therapies, phototherapy, chemotherapy, radiation, and bone marrow transplantation.<|end_of_text|>']

In [29]:
# Comments:
# Already only 150 Abstracts are used. In future, I will add more...